In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from datetime import datetime

from threading import Thread
import IPython
import IPython.display

import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import LSTM, LeakyReLU ,Flatten , Dense,Dropout
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

import matplotlib.pyplot as plt
from profiling import cumTimer , simple_timer

In [ ]:
print(tf.__version__)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
tf.test.is_built_with_cuda()

# Import Data

In [ ]:
InputData = np.genfromtxt('Data/InputDataBenchmark.csv',delimiter=",")
t = np.genfromtxt('Data/timeBenchmark.csv',delimiter=",")
Data = np.genfromtxt('Data/OutputDataBenchmark.csv',delimiter=",")
print(f"OutData: %s, \nInputData: %s, \nTime: %s" %(np.shape(Data),np.shape(InputData),np.shape(t)))

In [ ]:
example_ind = 3
plt.plot(t,Data[:,example_ind])
plt.grid()

# Split The Data

You'll use a (70%, 20%, 10%) split for the training, validation, and test sets. Note the data is not being randomly shuffled before splitting. This is for two reasons:

    It ensures that chopping the data into windows of consecutive samples is still possible.
    It ensures that the validation/test results are more realistic, being evaluated on the data collected after the model was trained.


In [ ]:
n = Data.shape[1]
train_df = Data[:, 0:int(n*0.7):]
meta_train_df = InputData.T[:, 0:int(n*0.7):]

val_df = Data[:,int(n*0.7):int(n*0.9):]
meta_val_df = InputData.T[:,int(n*0.7):int(n*0.9):]

test_df = Data[:,int(n*0.9):]
meta_test_df = InputData.T[:,int(n*0.9):]

num_features = Data.shape[1]


# Normalize Data

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [ ]:
plt.plot(t,train_df[:,example_ind])


# Data Windowing

In [ ]:
from window import WindowGenerator

In [ ]:
OUT_STEPS = 30
multi_window = WindowGenerator(input_width=30,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               batch_size = 200,
                               md_train_df =meta_train_df, md_test_df =meta_test_df,md_val_df=meta_val_df)

multi_window.plotexample()
multi_window

In [ ]:
multi_window.make_dataset(data=train_df,added_data=meta_train_df).element_spec

# FeedBack Model

In [ ]:
from network import FeedBack

In [ ]:
feedback_model = FeedBack(units=64, out_steps=OUT_STEPS)

# Observe Shapes

In [ ]:
prediction, *states = feedback_model.warmup(multi_window.example[0])
print('Input shape:', multi_window.example[0].shape)
print('Prediction shape:', prediction.shape)

## Output Shape

In [ ]:
%%time
print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

In [ ]:
feedback_model.load_weights(f"Models/{models[-1]}")


## Plot Example

In [ ]:
multi_window.plotexample(feedback_model)

## Plot Specific

In [ ]:
temp = multi_window.plotCase(train_df[:,3],t,feedback_model)

# Compile And Fit Model

In [ ]:
def compile_and_fit(model, name, window, patience=10, MAX_EPOCHS=500, record=False, load=None):
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y-%H-%M")
    if load is not None:
        model.load_weights(f"Models/{load}")
        NAME = load + "AR@"+str(MAX_EPOCHS)+dt_string
    else:
        NAME = name + "@"+str(MAX_EPOCHS)+"@"+dt_string

    filename = os.path.join("Models",  NAME + '.h5')

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min')
    if record == True:
        tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
        checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=0,
                                     save_best_only=True, mode='min', save_weights_only=True)
        callbacks = [early_stopping, tensorboard, checkpoint]
    else:
        callbacks = [early_stopping]

    model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(),
                  metrics=[tf.metrics.MeanAbsoluteError()])
    
    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=callbacks)
    return history


val_performance = {}
performance = {}


In [ ]:
multi_val_performance = {}
multi_performance = {}

In [ ]:
feedback_model.summary()

In [ ]:
models = os.listdir("Models")
print(models[-1])

In [ ]:
%%time
# mirrored_strategy = tf.distribute.MirroredStrategy()
# with mirrored_strategy.scope():
feedback_model = FeedBack(units=101, out_steps=OUT_STEPS)
print(feedback_model(multi_window.example[0]).shape)
history = compile_and_fit(feedback_model,
                        name="LSTM_AR100_Dropout",
                        window =multi_window, 
                        patience =20, 
                        MAX_EPOCHS=50,
                        record = True,
                        load =None
                        )

IPython.display.clear_output()

multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val)
multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test, verbose=0)



# Visualize Results

In [ ]:
multi_window.plotexample(feedback_model)

In [ ]:
startTime = 250
exampleCase = 1
CaseParameters = InputData[exampleCase]
# print("W: {}\nZ: {}\nX0: {}\nV0: {}".format(*CaseParameters))
pred = multi_window.plotCase( data=val_df[:, exampleCase],timesteps= t,metadata=meta_val_df[:, exampleCase], model=feedback_model)

In [ ]:
feedback_model.save_weights("Models/BenchmarkAR.h5")